In [1]:
import torch 
import torch.nn as nn

In [ ]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True        # (batch, seq_len, input_dim)
        )

        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x, h0=None, c0 = None):
        # x: (batch, seq_len, input_dim)
        if h0 is None or c0 is None:
            out, (hn, cn) = self.lstm(x)    # out : (batch, seq_len, hidden_dim)
        else:
            out, (hn, cn) = self.lstm(x, (h0, c0))

        last_out = out[:, -1 ,:]            # (batch, hidden_dim)
        y = self.fc(last_out)               # (batch, output_dim)
        return y, (hn,cn)
    


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

input_dim = 1
hidden_dim = 32
num_layers = 1
output_dim = 1

model = SimpleLSTM(input_dim, hidden_dim, num_layers, output_dim).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

X_train = X.to(device)
Y_train = y.to(device)

for epoch in range(100):
    model.train()
    optimizer.zero_grad()

    y_pred, _ = model(X_train)
    loss = criterion(y_pred, y_train)
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()

    if(epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}", Loss = {loss.item():.4f})